# Executing Job on Pasqal hardware

In [ ]:
import numpy as np
from threading import Thread

from qat.core import Schedule, Variable
from qat.core.variables import heaviside
from qat.qpus import RemoteQPU

Simulation on Pasqal hardware and simulation software can be performed using [Pulser](https://pulser.readthedocs.io/). This notebook first presents how to generate a `Job` that can be executed on Pasqal hardware, before showing how to execute it using Pasqal hardware. 

## Generating a Job to execute on Pasqal hardware

In [ ]:
from pulser import Pulse, Sequence, Register
from pulser.waveforms import CustomWaveform
from pulser.devices import MockDevice, AnalogDevice
from pulser_myqlm import IsingAQPU, FresnelQPU, FresnelDevice
from pulser_myqlm.myqlmtools import are_equivalent_schedules

On Pulser we can solve problems of shape 
$$ H = \hbar \sum_i \frac{\Omega(t)}{2}(\cos(\phi) \sigma_i^x - \sin(\phi) \sigma_i^y) - \frac{\delta(t)}{2}\sigma_i^z + \frac{1}{2}\sum_{i\neq j}U_{ij}n_i n_j$$
with $\sigma_i^x$, $\sigma_i^y$, $\sigma_i^z$ the Pauli operators $X$, $Y$, $Z$ applied on qubit $i$ and $n_i = \frac{1+\sigma_i^z}{2}$.

This Hamiltonian is named the <u>Ising Hamiltonian</u>. It is composed of a time-independent part, $\frac{1}{2}\sum_{i\neq j}U_{ij}n_i n_j$, and the rest of the terms that make a time-dependent Hamiltonian. In Pulser, this Hamiltonian is generated via a `Sequence` object. It is initialized by a `Device` and a `Register` that define the coefficients $U_{ij}$. Then, `Pulse`s are added to this `Sequence` to generate the time-dependent terms of this Hamiltonian. You can find more information about this Ising Hamiltonian in the [Pulser documentation](https://pulser.readthedocs.io/en/stable/review.html).  

The [pulser-myqlm package](https://github.com/pasqal-io/Pulser-myQLM) and its `IsingAQPU` class enables you to either generate step-by-step such an Ising Hamiltonian in a MyQLM `Schedule` or `Job`, or to convert a Pulser Sequence into a MyQLM `Schedule` or `Job` directly. We will introduce the two methods in the next two sections. 

### Step-by-step creation of MyQLM Schedule and Job using IsingAQPU

#### Defining an IsingAQPU

On Pulser, any simulation starts by defining a `Device` and a `Register`. Any `IsingAQPU` should be defined by these two objects. It can also be defined by a `Sequence` directly, this is explained in the next section.

`IsingAQPU` describes a specific AQPU on which only pulses applied on a `Rydberg.Global` channel can be run. This generates the Ising Hamiltonian described above. 

Pulser provides examples of Pasqal devices in `pulser.devices`. Any of these devices can be used for simulation purposes, yet simulation on real hardware should use the `FresnelDevice` defined in `pulser_myqlm.devices`. 

In real pulser devices such as `FresnelDevice`, only some `Register` can be implemented. They must be built from a `RegisterLayout` in the `calibrated_register_layouts` of the `FresnelDevice`. Let's have a look to one of these calibrated layouts of FresnelDevice:



In [ ]:
print(FresnelDevice.calibrated_register_layouts)
fresnel_layout = FresnelDevice.calibrated_register_layouts[
    "TriangularLatticeLayout(61, 5.0µm)"
]
fresnel_layout.draw()

The registers must be a subset of the calibrated layouts of `FresnelDevice`. They must be defined using the `define_register` method of the layouts. You can read more about registers and layouts [in the pulser documentation](https://pulser.readthedocs.io/en/stable/tutorials/reg_layouts.html).

Let's define a Register of triangular shape with sites 26, 35 and 30. The sites are spaced by 5µm.

In [ ]:
register = fresnel_layout.define_register(26, 35, 30)
register.draw()

Let's finally build an `IsingAQPU` with the `FresnelDevice` and the triangular register.

This `IsingAQPU` defines the interaction matrix between each atom, described by the matrix $U_{ij}$ in the Ising Hamiltonian. This interaction matrix can be accessed via the property `c6_interactions` of the `IsingAQPU`. Since the distances between each atom is the same, the interaction coefficients are expected to be the same as well.

In [ ]:
aqpu = IsingAQPU(FresnelDevice, register)
# Check qpu properties
print(
    "Distances (in um):\n", aqpu.distances, "\n"
)  # symetric matrix, for each qubit two qubits are at 4um distance and one at 4 * sqrt(2) um
print("Interactions (in rad/us):\n", aqpu.c6_interactions)  # Interaction matrix U_{ij}

#### Generating time-dependent Ising Hamiltonians

In the Ising Hamiltonian, the parameters of the time-dependent part of the Hamiltonian are $\Omega$, $\delta$ and $\phi$. They are the amplitude, detuning and phase of a `Pulse` in Pulser. The amplitude and the detuning can be time-dependent, whereas the phase is constant. Let's start by defining time-dependent amplitude and detuning:

In [ ]:
t_variable = Variable("t")  # in ns
u_variable = Variable("u")  # parameter
omega_t = t_variable / 100  # in rad/us
delta_t = (1 - t_variable + u_variable) / 100  # in rad/us

We can then compute the Ising Hamiltonians associated to pulses defined by $\Omega$, $\delta$, $\phi$ using `IsingAQPU.hamiltonian`.

- Here is an Hamiltonian associated with a pulse of constant amplitude $\Omega=1rad/\mu s$, zero detuning and phase.
It is equivalent to apply a X gate on each qubit, and ZZ gates whose coefficients are defined from the Van der Waals interactions between the atoms.

In [ ]:
t1 = 100  # in ns
H1 = aqpu.hamiltonian(1, 0, 0)  # omega(t)=1, delta(t)=0, phi=0
print(H1)  # in rad/us

- If now the phase is equal to $\pi / 2$, we no longer add a X gate on each qubit but a Y gate.

In [ ]:
t2 = 100  # in ns
H2 = aqpu.hamiltonian(1, 0, np.pi / 2)  # omega(t)=1, delta(t)=0, phi=pi/2
print(H2)  # in rad/us

- It is also possible to define time-dependent pulses, varying in amplitude and detuning. See that the coefficients in front the ZZ operators are unchanged.

In [ ]:
t0 = 100  # in ns
H0 = aqpu.hamiltonian(omega_t, delta_t, 0)  # omega(t)=omega_t, delta(t)=delta_t, phi=0
print(H0)  # in rad/us

Note that the output of `IsingAQPU.hamiltonian` is an Hamiltonian in $rad/\mu s$. It has to be multiplied by $\hbar$ to be in energy units.

#### Making a Schedule and a Job of Ising Hamiltonians

The Hamiltonian is implemented as the drive of a `Schedule` object. You also have to define the duration of the evolution of this `Schedule` object. In Pulser, the duration of Pulses are defined as integers in nanoseconds.

A `Schedule` can be implemented on the `IsingAQPU` if the coefficients in front of the ZZ operators match the Van der Waals interactions. Therefore, when using hamiltonians defined with `IsingAQPU`, the sum of the drive coefficients should be equal to 1 along the duration of the `Schedule`.

In [ ]:
schedule0 = Schedule(drive=[(1, H0)], tmax=t0)
schedule1 = Schedule(drive=[(1, H1)], tmax=t1)
schedule2 = Schedule(drive=[(1, H2)], tmax=t2)

In [ ]:
schedule = schedule0 | schedule1 | schedule2
print(schedule)

Here is an equivalent definition of the former `Schedule`, using `heaviside` explicitly instead of temporal compositions.

In [ ]:
schedule_sum = Schedule(
    drive=[
        [heaviside(t_variable, 0, t0), H0],
        [heaviside(t_variable, t0, t1 + t0), H1],
        [heaviside(t_variable, t1 + t0, t1 + t0 + t2), H2],
    ],
    tmax=t1 + t0 + t2,
)
print(schedule_sum)

In [ ]:
are_equivalent_schedules(schedule, schedule_sum)

A `Job` can be created from a `Schedule` via the `to_job` method. Read more regarding this method [in the MyQLM documentation](https://myqlm.github.io/04_api_reference/module_qat/module_core/schedule.html#qat.core.Schedule.to_job).

### Converting a Pulser Sequence into a Schedule or Job

The class method `from_sequence` of `IsingAQPU` creates an `IsingAQPU` instance having directly the `Register` and `Device` of the `Sequence`.

The class methods `IsingAQPU.convert_sequence_to_schedule` and `IsingAQPU.convert_sequence_to_job` also performs the direct conversion from a `Sequence` to a `Schedule` or `Job`. 

For instance, the `Schedule` could have also been built from a Pulser `Sequence` by adding the pulses described above.

In [ ]:
seq = Sequence(register, FresnelDevice)
seq.declare_channel("ryd_glob", "rydberg_global")
seq.add(
    Pulse(
        CustomWaveform([omega_t(t=ti) for ti in range(t0)]),
        CustomWaveform(
            [delta_t(t=ti, u=0) for ti in range(t0)]
        ),  # no parametrized sequence for the moment
        0,
    ),
    "ryd_glob",
)  # corresponds to H0
seq.add(Pulse.ConstantPulse(t1, 1, 0, 0), "ryd_glob")  # corresponds to H1
seq.add(Pulse.ConstantPulse(t2, 1, 0, np.pi / 2), "ryd_glob")  # corresponds to H2

In [ ]:
schedule_from_seq = IsingAQPU.convert_sequence_to_schedule(seq)

In [ ]:
are_equivalent_schedules(schedule_from_seq, schedule(u=0))

The pulses of the `Sequence` are the instructions sent to the hardware. If you want a model of the behaviour of the hardware, you should convert while setting the `modulation` argument to `True`. You can learn more about modulation [in the Pulser documentation](https://pulser.readthedocs.io/en/stable/tutorials/output_mod_eom.html).

In [ ]:
# Samples of the Sequence are in full line, modulated samples in hatched
# mod_schedule_from_seq contains the modulated samples of the analog_seq
seq.draw()
mod_schedule_from_seq = IsingAQPU.convert_sequence_to_schedule(seq, True)
print(mod_schedule_from_seq)

It is also possible to convert the Pulser `Sequence` directly into a job using `IsingAQPU.convert_sequence_to_job`. You can define the number of shots of the job in the attribute `nbshots`. The default value asks for the maximum number of shots the device can take.

In [ ]:
job_from_seq = IsingAQPU.convert_sequence_to_job(seq, nbshots=1000, modulation=True)
print(job_from_seq)

## Simulating a Job on Pasqal hardware and simulation tools

Any `Job` implementing an Ising Hamiltonian can be simulated using a local or remote QPU of Qaptiva Access by using its `submit_job` method. The only condition on this QPU is that it must be able to simulate [Analog Jobs](https://myqlm.github.io/02_user_guide/01_write/02_analog_schedule/03_an_jobs.html).

A Job encapsulating a serialized Pulser `Sequence` under the key "abstr_dict" of the dictionary `Job.schedule._other` can be simulated using Pasqal hardware and simulation tools.

`IsingAQPU` has a `submit_job` method that can be used to simulate the sequence either using `pulser_simulation` or another qpu. To use `pulser_simulation`, simply set the QPU to `None`:

In [ ]:
# Create an IsingAQPU
simulation_aqpu = IsingAQPU.from_sequence(seq, qpu=None)
# Simulate the sequence
# If job was converted from the sequence using modulation=True,
# modulated samples of the sequence are used for the simulation
result = simulation_aqpu.submit_job(job_from_seq, n_samples=1000)
print(result)

### FresnelQPU

`FresnelQPU` is a `QPUHandler` that interfaces a QPU. It should be instantiated on the same network as the QPU, using its public address. 

In [ ]:
base_uri = "http://10.8.15.211:4300/api"
fresnel_qpu = FresnelQPU(base_uri=base_uri, version="v1")
print("The QPU is operational: ", fresnel_qpu.is_operational)

If this QPU is operational, you can run it on a Qaptiva server via the `serve` method. Any user can then access this QPU via a `RemoteQPU`:

In [ ]:
# Deploy the QPU on a Qaptiva server
def deploy_qpu(qpu):
    qpu.serve(1234, "localhost")


server_thread = Thread(target=deploy_qpu, args=(fresnel_qpu,))
server_thread.start()
# Access it remotly
qpu = RemoteQPU(1234, "localhost")

To submit a MyQLM job to `FresnelQPU`, you need to furnish a serialized Pulser `Sequence` in `Job.schedule._other`. This `Sequence` must use the device `Fresnel` as its device. This device `Fresnel` is furnished with the package pulser-myqlm in the module "devices", but is also an attribute of the `FresnelQPU` class.


In [ ]:
assert fresnel_qpu.device == FresnelDevice
fresnel_qpu.submit_job(IsingAQPU.convert_sequence_to_job(seq))